# Python decorators


## Sources

All good articles on Python decorators look fundamentally the same. One of the best examples is linked below, and if you can you should read it.


https://realpython.com/primer-on-python-decorators/


## What are decorators?


_Decorators_ are a shorthand Python syntax to help programmers - that's you - work with functions that operate on functions. These are called _higher-order functions_. Yes, this means they actually are an example of _syntactic sugar_.


### Why would I want a higher-order function?


Higher-order functions can be a way to reason rigorously about computation and there are entire fields of math, such as _lambda calculus_ that use them in this way. But Python isn't a language for expressing proofs, so by the time we're writing Python we probably aren't concerned with higher-order functions for their mathematical properties.


Python has many higher-order functions, presented as decorators, either built in or in the standard library. The `@property` decorator allows a function - really a set of functions - to behave like an attribute. `@staticmethod` and `@classmethod` allow class methods to relate to their classes in special ways. `functools.lru_cache` adds caching to a function. `@contextlib.contextmanager` allows programmers to write _context managers_ using _generator functions_. Libraries like **flask** use decorators like `@app.get()` to associate Python functions with particular routes - combinations of _HTTP verbs_ and URL paths - in the resulting website. These should provide a broad overview of the sorts of things that higher-order functions, and therefore decorators, can do in Python.


## The sugar-free alternative


Any Python function with the right signature can be used as a decorator. Put another way, decorators are just a special way of calling regular Python functions. So, what is the un-special way of doing the same thing?


This will also help illustrate the conventions that will be used to help us keep track of things in this notebook.


### Minimal working sugar-free example


In [66]:
def factory_function(function_to_wrap):
    print("Starting factory function")

    def wrapper_func():
        print("Inside wrapper before calling wrapped function")
        function_to_wrap()
        print("Inside wrapper after calling wrapped function")
    print("Finishing factory function by returning the new wrapper")
    return wrapper_func


def original_function():
    print("Inside original function")


original_function = factory_function(original_function)

Starting factory function
Finishing factory function by returning the new wrapper


Here is the output of the above code block on my machine:

```python console
Starting factory function
Finishing factory function by returning the new wrapper
```


In [67]:
original_function()

Inside wrapper before calling wrapped function
Inside original function
Inside wrapper after calling wrapped function


Here is the output of the above code block on my machine:

```python console
Inside wrapper before calling wrapped function
Inside original function
Inside wrapper after calling wrapped function
```


### What just happened?


In this example, `factory_function()` is operating as a decorator. I created `original_function()` but then replaced it with the return value of `factory_function()`.


#### Important things to note:


- `factory_function()` - our decorator - only runs one time.
- The factory function returns an _inner function_ called `wrapper_func()`.
- `wrapper_func()` retains a reference to the original function through _closure_.
- `wrapper_func()` replaces `original_function()`.
- When the user calls the decorated `original_function()` it is `wrapper_func()` that's called each time.
- `wrapper_func()` then goes on to call `original_function()`


#### Python def is combined initializer and assignment statement


Both the sugar-free example above and the special decorator syntax work because the def statement isn't as special as it might first appear.

It's common to see Python statements like the following:

```python
my_variable = ["foo", "bar", "baz"]
```

When we see this, we understand that this single statement is really dealing with two things: The variable named `my_variable` and the list object `["foo", "bar", "baz"]`. We could go on to operate on these things separately. We might change the list, or assign it to an additonal variable name. We might reassign the variable name to a float, a string, a new list, or anything else.

The python `def` statement works just like the assignment statement above, it only looks a little different. `def foo()` creates a function - which in Python is just one type of object - and assigns that function to the variable `foo`. And we're free to pass around that function object and reassign that variable name, just like we can in the list example.

So, we pass the function object into the factory function, and reassign the variable name to the result.


## Minimal working sugared example


In [68]:
def decorator_function(function_to_decorate):
    print("Starting decorator function")

    def wrapper_function():
        print("Inside wrapper function before calling wrapped function")
        function_to_decorate()
        print("Inside wrapper function before calling wrapped function")
    print("Finishing decorator function by returning new wrapper")
    return wrapper_function

In [69]:
@decorator_function
def function_to_decorate():
    print("Inside function to decorate")

Starting decorator function
Finishing decorator function by returning new wrapper


Here are the results of running the above code block on my machine:

```python console
Starting decorator function
Finishing decorator function by returning new wrapper
```


In [70]:
function_to_decorate()

Inside wrapper function before calling wrapped function
Inside function to decorate
Inside wrapper function before calling wrapped function


Here are the results of running the above code block on my machine:

```python console
Inside wrapper function before calling wrapped function
Inside function to decorate
Inside wrapper function before calling wrapped function
```


### What is the special syntax buying us?


If you inspect the above output you'll see that it's fundamentally the same as the example above that doesn't use the special decorator syntax. That's the ideal for syntactic sugar. The programmer could always choose not to use it. But the decorator syntax helps us communicate our intentions, makes the code easier to read, and saves repettition, so we should generally use it.


Surprisingly, the decorator syntax only replaces one line in the sugar-free example, this one:

```python
original_function = factory_function(original_function)
```

In what ways is it helping? Well, this variable reassignment could be done anywhere, but the decorator syntax really only works above the definition of the function to be decorated. This minimizes spooky action at a distance. Additionally, particularly when applying decorators that themselves accept arguments, the non-sugared syntax looks highly unusual for Python and can be difficult to follow. For example, it creates situations where parenthesis immediately follow parenthessis, like this:

```python
decorated_function = decorator_that_takes_arguments(*args_to_decorator)(original_function)
```

This works because the function call `decorator_that_takes_arguments(*args_to_decorator)` returns a function, which the second set of parenthesis are immediately calling. Some languages, like JavaScript, embrace this sort of syntax but Python really does not.


## Helpful analogy

Decorators are functions that themselves accept and return functions. This can be difficult to remember. Think of it like decorating a Christmas tree. You have an original object which is just a log you've brought inside for some reason. Then you apply decorations to turn it into a centerpiece of festivities. The decorator - a person - performs the transformation but isn't a part of either the original or the decorated tree and doesn't have to stick around.


## Some things that don't work


You might wonder if it's really necessary to bother with closures and returning an inner function. Even if you don't, forgetting this convolution is one of the most common mistakes when writing a decorator in a hurry. Let's try it and see what happens.


In [71]:
def hasty_decorator(function_to_decorate):
    print("Inside decorator function before calling original function")
    function_to_decorate()
    print("Inside decorator funcction after calling original function")

In [72]:
@hasty_decorator
def base_function():
    print("inside original function")

Inside decorator function before calling original function
inside original function
Inside decorator funcction after calling original function


Here are the results of running the above code block on my machine:

```python console
Inside decorator function before calling original function
inside original function
Inside decorator funcction after calling original function
```


In [73]:
base_function()

TypeError: 'NoneType' object is not callable

Here are the results of running the above code block on my machine:

```python traceback
---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
Cell In[65], line 1
----> 1 base_function()

TypeError: 'NoneType' object is not callable
```


This doesn't work because the function being used as a decorator doesn't return a function. Without a return statement it returns `None` by default. But even if I returned the results of calling the original function, this most likely wouldn't be a function either.


## Closures and learning to live with them

## Decorating functions that return a value

## Decorating functions that take arguments

## Decorators that take arguments

## Stacking decorators

## Decorating classes

## Classes as decorators